In [1]:
import re
import pandas as pd
import pickle

In [2]:
file1 = open('1645528771.result/1645528771.fas.1.clstr.sorted', 'r')
Lines = file1.readlines()

In [3]:
LineIndex = 0
cd_hit_dict = {}
while LineIndex < len(Lines):
    FF = re.search(r'^(>Cluster.*)', Lines[LineIndex])
    if FF != None:
        LineIndex += 1
        li = []
        dkey = ''
        while LineIndex < len(Lines):
            KK = re.search(r'^(.*\*)', Lines[LineIndex])
            if KK != None:
                dkey = KK.group()
                dkey = re.sub(r'.*>','',dkey)
                dkey = re.sub(r'\.\.\. \*','',dkey)
            else:
                KK = re.search(r'^(.*\.\.\.)', Lines[LineIndex])
                dval = KK.group()
                dval = re.sub(r'.*>','',dval)
                dval = re.sub(r'\.\.\.','',dval)
                li.append(dval)
            if(LineIndex == len(Lines)-1):
                break
            FF = re.search(r'^(>Cluster.*)', Lines[LineIndex + 1])
            if (FF == None):
                LineIndex += 1
            else:
                break
        cd_hit_dict.update({dkey: li})
    LineIndex += 1

In [4]:
dip_li = []
for k in cd_hit_dict.keys():
    if(k.startswith('DIP')):
        if(not(cd_hit_dict[k])):
            dip_li.append(k)
        elif(all(protein.startswith('DIP') for protein in cd_hit_dict[k])):
            dip_li.append(k)
            dip_li.extend(cd_hit_dict[k])
len(dip_li)

532

In [5]:
with open('dip_int_and_dict.pkl', 'rb') as f:
    dip_interactions, seq_dict = pickle.load(f)

dip_selected = {x : seq_dict[x] for x in dip_li}

In [6]:
p1 = []
p2 = []
for i in range(len(dip_interactions)):
    if ((dip_interactions.loc[i][0] in dip_selected) or (dip_interactions.loc[i][1] in dip_selected)):
        p1.append(dip_interactions.loc[i][0])
        p2.append(dip_interactions.loc[i][1])

final_dip_interactions_selected = pd.DataFrame({'Protein A': p1, 'Protein B': p2})

In [7]:
final_dip_interactions_selected

,Protein A,Protein B
0,DIP-617N,DIP-58525N
1,DIP-617N,DIP-58999N
2,DIP-617N,DIP-61424N
3,DIP-582N,DIP-570N
4,DIP-582N,DIP-1090N
...,...,...
928,DIP-61628N,DIP-56422N
929,DIP-40642N,DIP-35754N
930,DIP-52332N,DIP-56130N
931,DIP-61775N,DIP-61776N


In [8]:
len(cd_hit_dict)

7877

In [9]:
len(dip_selected)

532

In [10]:
with open('diplr.pkl', 'wb') as f:
    pickle.dump([final_dip_interactions_selected, seq_dict], f)